## Preparing for Data Scraping

Public Data Source: https://www.seoul.go.kr/coronaV/coronaStatus.do

In [583]:
import pandas as pd
import numpy as np

In [584]:
import requests

In [585]:
url = "https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1"
#url = f"{url}&columns%5B0%5D%5Bdata%5D=0&columns%5B0%5D%5Bname%5D=&columns%5B0%5D%5Bsearchable%5D=true&columns%5B0%5D%5Borderable%5D=true&columns%5B0%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B0%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B1%5D%5Bdata%5D=1&columns%5B1%5D%5Bname%5D=&columns%5B1%5D%5Bsearchable%5D=true&columns%5B1%5D%5Borderable%5D=true&columns%5B1%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B1%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B2%5D%5Bdata%5D=2&columns%5B2%5D%5Bname%5D=&columns%5B2%5D%5Bsearchable%5D=true&columns%5B2%5D%5Borderable%5D=true&columns%5B2%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B2%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B3%5D%5Bdata%5D=3&columns%5B3%5D%5Bname%5D=&columns%5B3%5D%5Bsearchable%5D=true&columns%5B3%5D%5Borderable%5D=true&columns%5B3%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B3%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B4%5D%5Bdata%5D=4&columns%5B4%5D%5Bname%5D=&columns%5B4%5D%5Bsearchable%5D=true&columns%5B4%5D%5Borderable%5D=true&columns%5B4%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B4%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B5%5D%5Bdata%5D=5&columns%5B5%5D%5Bname%5D=&columns%5B5%5D%5Bsearchable%5D=true&columns%5B5%5D%5Borderable%5D=true&columns%5B5%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B5%5D%5Bsearch%5D%5Bregex%5D=false&columns%5B6%5D%5Bdata%5D=6&columns%5B6%5D%5Bname%5D=&columns%5B6%5D%5Bsearchable%5D=true&columns%5B6%5D%5Borderable%5D=true&columns%5B6%5D%5Bsearch%5D%5Bvalue%5D=&columns%5B6%5D%5Bsearch%5D%5Bregex%5D=false&order%5B0%5D%5Bcolumn%5D=0&order%5B0%5D%5Bdir%5D=desc"
url = f"{url}&start=0&length=100"
#"&search%5Bvalue%5D=&search%5Bregex%5D=true&_=1628930508535"
url

'https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw=1&start=0&length=100'

In [586]:
response = requests.get(url)
data_json = response.json()
# pd.DataFrame(data_json['data'])


In [587]:
records_total = data_json['recordsTotal']
records_total

42086

In [588]:
end_page = round(records_total/ 100) + 1
end_page                                        # used in get_multi_page_list function

422

In [589]:
data = data_json["data"]

In [590]:
pd.DataFrame(data)

,0,1,2,3,4,5,6
0,<p class='corona19_no'>72086</p>,226846,2021-08-16,동대문구,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>72085</p>,226698,2021-08-16,관악구,-,기타 확진자 접촉,<b class=''>-</b>
2,<p class='corona19_no'>72084</p>,226685,2021-08-16,강남구,-,감염경로 조사중,<b class=''>-</b>
3,<p class='corona19_no'>72083</p>,226669,2021-08-16,마포구,-,기타 확진자 접촉,<b class=''>-</b>
4,<p class='corona19_no'>72082</p>,226644,2021-08-16,강남구,미국,해외유입,<b class=''>-</b>
...,...,...,...,...,...,...,...
95,<p class='corona19_no'>71991</p>,226153,2021-08-16,도봉구,-,감염경로 조사중,<b class=''>-</b>
96,<p class='corona19_no'>71990</p>,226466,2021-08-16,영등포구,-,기타 확진자 접촉,<b class=''>-</b>
97,<p class='corona19_no'>71989</p>,226474,2021-08-16,영등포구,-,타시도 확진자 접촉,<b class=''>-</b>
98,<p class='corona19_no'>71988</p>,226352,2021-08-16,은평구,-,기타 확진자 접촉,<b class=''>-</b>


## Function for scraping one pages

In [591]:
def get_seoul_covid19_100(page_no):
    start_no = (page_no - 1) * 100
    url = f"https://news.seoul.go.kr/api/27/getCorona19Status/get_status_ajax.php?draw={page_no}"
    url = f"{url}&start={start_no}&length=100"
    response = requests.get(url)
    data_json = response.json()
    return data_json

In [592]:
# data unavailable
get_seoul_covid19_100(-1)

{'draw': -1, 'recordsTotal': 42086, 'recordsFiltered': 42086, 'data': []}

## Python code for combining all pages

In [593]:
# to prevent network demage
import time
# tqdm: process check
from tqdm import trange

In [594]:
def get_multi_page_list(start_page, end_page):
    page_list = []
    for page_no in trange(start_page, end_page + 1):
        one_page = get_seoul_covid19_100(page_no)
        # break when page is empty
        if len(one_page['data']) > 0:
            one_page = pd.DataFrame(one_page['data'])
            page_list.append(one_page)
        else:
            return page_list
    return page_list
        
    


In [595]:
end_page

422

In [596]:
# Test Code
start_page = 1
end_page = round(records_total/ 100) + 1
# end_page = 205
page_list = get_multi_page_list(start_page, end_page)
page_list[:1]

100%|█████████▉| 421/422 [09:18<00:01,  1.33s/it]


[                                   0       1           2     3   4  \
 0   <p class='corona19_no'>72086</p>  226846  2021-08-16  동대문구   -   
 1   <p class='corona19_no'>72085</p>  226698  2021-08-16   관악구   -   
 2   <p class='corona19_no'>72084</p>  226685  2021-08-16   강남구   -   
 3   <p class='corona19_no'>72083</p>  226669  2021-08-16   마포구   -   
 4   <p class='corona19_no'>72082</p>  226644  2021-08-16   강남구  미국   
 ..                               ...     ...         ...   ...  ..   
 95  <p class='corona19_no'>71991</p>  226153  2021-08-16   도봉구   -   
 96  <p class='corona19_no'>71990</p>  226466  2021-08-16  영등포구   -   
 97  <p class='corona19_no'>71989</p>  226474  2021-08-16  영등포구   -   
 98  <p class='corona19_no'>71988</p>  226352  2021-08-16   은평구   -   
 99  <p class='corona19_no'>71987</p>  226853  2021-08-16  서대문구   -   
 
              5                  6  
 0     감염경로 조사중  <b class=''>-</b>  
 1    기타 확진자 접촉  <b class=''>-</b>  
 2     감염경로 조사중  <b class=''>-</b> 

In [597]:
df_all = pd.concat(page_list)
df_all.shape

(42086, 7)

In [598]:
cols = ['Number','Patient','Confirmed Date','Area','Travel History','Transmission Route','Status']

In [599]:
df_all.columns = cols

In [600]:
df_all.head()

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,<p class='corona19_no'>72086</p>,226846,2021-08-16,동대문구,-,감염경로 조사중,<b class=''>-</b>
1,<p class='corona19_no'>72085</p>,226698,2021-08-16,관악구,-,기타 확진자 접촉,<b class=''>-</b>
2,<p class='corona19_no'>72084</p>,226685,2021-08-16,강남구,-,감염경로 조사중,<b class=''>-</b>
3,<p class='corona19_no'>72083</p>,226669,2021-08-16,마포구,-,기타 확진자 접촉,<b class=''>-</b>
4,<p class='corona19_no'>72082</p>,226644,2021-08-16,강남구,미국,해외유입,<b class=''>-</b>


In [601]:
df_all.shape

(42086, 7)

## Data Preprocessing

### Number and Status

In [602]:
import re
def extract_number(num_string):
    if type(num_string) == str:
        num_string = num_string.replace("corona19", "")
        num = re.sub("[^0-9]", "", num_string)
        num = int(num)
        return num
    else:
        return num_string

In [603]:
num_string = "<p class='corona19_no'>7625</p>"
extract_number(num_string)

7625

In [604]:
df_all['Number'] = df_all["Number"].map(extract_number)

In [605]:
df_all.head()

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,72086,226846,2021-08-16,동대문구,-,감염경로 조사중,<b class=''>-</b>
1,72085,226698,2021-08-16,관악구,-,기타 확진자 접촉,<b class=''>-</b>
2,72084,226685,2021-08-16,강남구,-,감염경로 조사중,<b class=''>-</b>
3,72083,226669,2021-08-16,마포구,-,기타 확진자 접촉,<b class=''>-</b>
4,72082,226644,2021-08-16,강남구,미국,해외유입,<b class=''>-</b>


In [606]:
def extract_hangeul(origin_text):
    subtract_text = re.sub("[^가-힣]", "", origin_text)
    return subtract_text

In [607]:
extract_hangeul("<b class='status1'>퇴원</b>")

'퇴원'

In [608]:
extract_hangeul("<b class='status2'>사망</b>")

'사망'

In [609]:
extract_hangeul("<b class=''>-</b>")

''

In [610]:
df_all["Status"].value_counts()

<b class='status1'>퇴원</b>    32800
<b class='status1'>-</b>      5177
<b class=''>-</b>             3767
<b class='status2'>-</b>       216
<b class='status2'>사망</b>      126
Name: Status, dtype: int64

In [611]:
df_all.loc[df_all["Status"].str.contains("퇴원"), "Status"] = "Recovered"
df_all.loc[df_all["Status"].str.contains("사망"), "Status"] = "Death"
df_all.loc[~df_all["Status"].str.contains("Recovered|Death"), "Status"] = "Quarantined"
df_all

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,72086,226846,2021-08-16,동대문구,-,감염경로 조사중,Quarantined
1,72085,226698,2021-08-16,관악구,-,기타 확진자 접촉,Quarantined
2,72084,226685,2021-08-16,강남구,-,감염경로 조사중,Quarantined
3,72083,226669,2021-08-16,마포구,-,기타 확진자 접촉,Quarantined
4,72082,226644,2021-08-16,강남구,미국,해외유입,Quarantined
...,...,...,...,...,...,...,...
81,30005,95804,2021-03-14,송파구,-,타시도 확진자 접촉,Recovered
82,30004,95889,2021-03-14,종로구,-,감염경로 조사중,Recovered
83,30003,95799,2021-03-14,중구,-,중구 소재 주점/음식점 관련,Recovered
84,30002,95872,2021-03-14,종로구,-,기타 확진자 접촉,Recovered


In [612]:
df_all.loc[df_all["Transmission Route"].str.contains("조사중"), "Transmission Route"] = "Under Inspection"
df_all.loc[df_all["Transmission Route"].str.contains("접촉|관련"), "Transmission Route"] = "Known"
df_all.loc[df_all["Transmission Route"].str.contains("해외"), "Transmission Route"] = "Overseas"
df_all.loc[~df_all["Transmission Route"].str.contains("Under Inspection|Known|Overseas"), "Transmission Route"] = "Unknown"
df_all

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,72086,226846,2021-08-16,동대문구,-,Under Inspection,Quarantined
1,72085,226698,2021-08-16,관악구,-,Known,Quarantined
2,72084,226685,2021-08-16,강남구,-,Under Inspection,Quarantined
3,72083,226669,2021-08-16,마포구,-,Known,Quarantined
4,72082,226644,2021-08-16,강남구,미국,Overseas,Quarantined
...,...,...,...,...,...,...,...
81,30005,95804,2021-03-14,송파구,-,Known,Recovered
82,30004,95889,2021-03-14,종로구,-,Under Inspection,Recovered
83,30003,95799,2021-03-14,중구,-,Known,Recovered
84,30002,95872,2021-03-14,종로구,-,Known,Recovered


In [613]:
df_all.loc[df_all["Area"].str.contains("종로구"), "Area"] = "Jongno-gu"
df_all.loc[df_all["Area"].str.contains("중구"), "Area"] = "Jung-gu"
df_all.loc[df_all["Area"].str.contains("용산구"), "Area"] = "Yongsan-gu"
df_all.loc[df_all["Area"].str.contains("성동구"), "Area"] = "Seongdong-gu"
df_all.loc[df_all["Area"].str.contains("광진구"), "Area"] = "Gwangjin-gu"
df_all.loc[df_all["Area"].str.contains("동대문구"), "Area"] = "Dongdaemun-gu"
df_all.loc[df_all["Area"].str.contains("중랑구"), "Area"] = "Jungnang-gu"
df_all.loc[df_all["Area"].str.contains("성북구"), "Area"] = "Seongbuk-gu"
df_all.loc[df_all["Area"].str.contains("강북구"), "Area"] = "Gangbuk-gu"
df_all.loc[df_all["Area"].str.contains("도봉구"), "Area"] = "Dobong-gu"
df_all.loc[df_all["Area"].str.contains("노원구"), "Area"] = "Nowon-gu"
df_all.loc[df_all["Area"].str.contains("은평구"), "Area"] = "Eunpyeong-gu"
df_all.loc[df_all["Area"].str.contains("서대문구"), "Area"] = "Seodaemun-gu"
df_all.loc[df_all["Area"].str.contains("마포구"), "Area"] = "Mapo-gu"
df_all.loc[df_all["Area"].str.contains("양천구"), "Area"] = "Yangcheon-gu"
df_all.loc[df_all["Area"].str.contains("강서구"), "Area"] = "Gangseo-gu"
df_all.loc[df_all["Area"].str.contains("구로구"), "Area"] = "Guro-gu"
df_all.loc[df_all["Area"].str.contains("금천구"), "Area"] = "Geumcheon-gu"
df_all.loc[df_all["Area"].str.contains("영등포구"), "Area"] = "Yeongdeungpo-gu"
df_all.loc[df_all["Area"].str.contains("동작구"), "Area"] = "Dongjak-gu"
df_all.loc[df_all["Area"].str.contains("관악구"), "Area"] = "Gwanak-gu"
df_all.loc[df_all["Area"].str.contains("서초구"), "Area"] = "Seocho-gu"
df_all.loc[df_all["Area"].str.contains("강남구"), "Area"] = "Gangnam-gu"
df_all.loc[df_all["Area"].str.contains("송파구"), "Area"] = "Songpa-gu"
df_all.loc[df_all["Area"].str.contains("강동구"), "Area"] = "Gangdong-gu"
df_all.loc[df_all["Area"].str.contains("타시도|기타"), "Area"] = "Another City"

In [614]:
file_name = 'covid_19_recent_data.csv'
df_all.to_csv(file_name, index=False)

In [615]:
pd.read_csv(file_name)

,Number,Patient,Confirmed Date,Area,Travel History,Transmission Route,Status
0,72086,226846,2021-08-16,Dongdaemun-gu,-,Under Inspection,Quarantined
1,72085,226698,2021-08-16,Gwanak-gu,-,Known,Quarantined
2,72084,226685,2021-08-16,Gangnam-gu,-,Under Inspection,Quarantined
3,72083,226669,2021-08-16,Mapo-gu,-,Known,Quarantined
4,72082,226644,2021-08-16,Gangnam-gu,미국,Overseas,Quarantined
...,...,...,...,...,...,...,...
42081,30005,95804,2021-03-14,Songpa-gu,-,Known,Recovered
42082,30004,95889,2021-03-14,Jongno-gu,-,Under Inspection,Recovered
42083,30003,95799,2021-03-14,Jung-gu,-,Known,Recovered
42084,30002,95872,2021-03-14,Jongno-gu,-,Known,Recovered
